In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [2]:
df_history = pd.read_csv("C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten/"
                                    "Thalia_history_ersteBereinigung.csv")

In [3]:
df_relevante_iids = pd.read_csv("C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten/" +
                                    "Thalia_transactions_zweiteBereinigung_iids_Validierung.csv")

##### Relevante Kunden und Hist mergen

In [4]:
df_h = pd.merge(df_history, df_relevante_iids, on = ["iid"])
df_h[["iid","attrtype", "text", "month_count"]].head(5)

,iid,attrtype,text,month_count
0,100011954,HY,|msg=NL_Mai_1|subject=St%C3%BCck+der+Stunde%3A...,0
1,100011954,HY,|msg=NL_Mai_2|subject=Online+Premiere+%22Kabal...,0
2,100011954,HY,|msg=NL_Corona_1|subject=Sondernewsletter%3A+J...,1
3,100011954,HY,|msg=NL_Programm_Spielzeit20_21|subject=Das+Pr...,1
4,100011954,HY,|msg=NL_Abschluss Saison 19_20|subject=Zum+End...,1


In [5]:
df_h[["iid","attrtype", "month_count"]].head(5)

,iid,attrtype,month_count
0,100011954,HY,0
1,100011954,HY,0
2,100011954,HY,1
3,100011954,HY,1
4,100011954,HY,1


##### Erstellung des "df_neueSpalte" für die Betrachtung sämtlicher Infos zwischen den "|"

In [6]:
neueSpalten = [ line.strip().split('|') for line in df_h["text"] ]

##### Ein neues DataFrame df_user_agent und eine neue Spalte "User agent", die noch sämtliche Informationen danach enthält

In [7]:
df_user_agent = pd.DataFrame()
df_user_agent["User Agent"] = df_h["text"].str.extract(r'userAgent=(.*)', expand=False)
df_user_agent["User Agent"] = df_user_agent["User Agent"].apply(lambda zeile: zeile.split("|")[0] if isinstance(zeile, str) else zeile)

In [8]:
df_user_agent.head()

,User Agent
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


##### Ermittlun aller iids die mindestens einmal mit Apple Mail/Gmail oder Proxy-Server eine Email geöffnet haben

In [9]:
df_user_agent["User Agent"].fillna("unknown", inplace=True)
df_h["User Agent"] = df_user_agent["User Agent"]
df_h.head()

,iid,attrtype,akey,date,text,month_count,User Agent
0,100011954,HY,107601295,2020-05-07 11:49:16,|msg=NL_Mai_1|subject=St%C3%BCck+der+Stunde%3A...,0,unknown
1,100011954,HY,108268529,2020-05-19 11:15:17,|msg=NL_Mai_2|subject=Online+Premiere+%22Kabal...,0,unknown
2,100011954,HY,108552325,2020-06-02 18:07:17,|msg=NL_Corona_1|subject=Sondernewsletter%3A+J...,1,unknown
3,100011954,HY,108668409,2020-06-17 13:13:00,|msg=NL_Programm_Spielzeit20_21|subject=Das+Pr...,1,unknown
4,100011954,HY,109328444,2020-06-25 16:52:03,|msg=NL_Abschluss Saison 19_20|subject=Zum+End...,1,unknown


In [10]:
irrelevante_iids = df_h[df_h["User Agent"].str.contains('Proxy|Apple Mail|Gmail', case=False)]["iid"].unique()
irrelevante_iids

array([100074164, 100074334, 100074740, 100076440, 100077574, 100081532,
       100086688, 100090142, 100094500, 100095830, 100096030, 100104100,
       100106042, 100110624, 100113202, 100122034, 100127080, 100127698,
       100128412, 100132768, 100139030, 100146782, 100154819, 100155545,
       100156117, 100159959, 100161211, 100162457, 100171937, 100174389,
       100183114, 100193728, 100201123, 100201267, 100211515, 100230944,
       100241333, 100242563, 100245941, 100247657, 100248875, 100256545,
       100262038, 100274332, 100276744, 100279344, 100280330, 100290122,
       100300172, 100302744, 100304874, 100309090, 100343874, 100352609,
       100357367, 100360041, 100372373, 100412492, 100415993, 100416328,
       100418184, 100423388, 100424543, 100426217, 100428193, 100428258,
       100429785, 100433284, 100441760, 100443613, 100444487, 100448151,
       100456245, 100457982, 100461063, 100482266, 100491620, 100497684,
       100499348, 100502928, 100513469, 100513501, 

In [11]:
relevant_iids = df_relevante_iids["iid"]

filtered_iids = [iid for iid in relevant_iids if iid not in irrelevante_iids]

In [12]:
df_filtered_iids = pd.DataFrame(filtered_iids, columns = ["iid"])
df_filtered_iids.head()

,iid
0,158646281
1,137986060
2,107208724
3,100778018
4,100679736


In [13]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten/'
filename = 'privatkunden.iid.ELPHI.ohneAppleEtc_Validierung.csv'
df_filtered_iids.to_csv(os.path.join(path, filename), index=False)

# Erweiterte Analysen zum Verständnis

##### Ergänzung duch Infos für welche Datenpunkte Probleme bei der Genauigkeit der Öffnungsrate besteht

In [8]:
df_user_agent["User Agent"].fillna("unknown", inplace=True)
df_user_agent_aufgeteilt = df_user_agent

# Spalten mit Ja/Nein-Werten erstellen
df_user_agent_aufgeteilt["unknown"] = df_user_agent['User Agent'].str.contains('unknown', case=False)
#df_user_agent_aufgeteilt['iOS-Nutzer'] = df_user_agent['User Agent'].str.contains('iOS', case=False)
#df_user_agent_aufgeteilt['OS-X-Nutzer'] = df_user_agent['User Agent'].str.contains('OS X|Macintosh', case=False)
df_user_agent_aufgeteilt['Apple-Mail-Nutzer'] = df_user_agent['User Agent'].str.contains('Apple Mail', case=False)
df_user_agent_aufgeteilt['Gmail-Nutzer'] = df_user_agent['User Agent'].str.contains('Gmail', case=False)
df_user_agent_aufgeteilt['Proxy-Server'] = df_user_agent['User Agent'].str.contains('Proxy', case=False)
df_user_agent_aufgeteilt.head()

,User Agent,unknown,Apple-Mail-Nutzer,Gmail-Nutzer,Proxy-Server
0,unknown,True,False,False,False
1,Apple Mail (OS X / Personal computer),False,True,False,False
2,unknown,True,False,False,False
3,Apple Mail (OS X / Personal computer),False,True,False,False
4,unknown,True,False,False,False


In [9]:
for col in df_user_agent.columns[1:]:
    print(df_user_agent[col].value_counts())

True     80292
False    30918
Name: unknown, dtype: int64
False    106187
True       5023
Name: Apple-Mail-Nutzer, dtype: int64
False    108781
True       2429
Name: Gmail-Nutzer, dtype: int64
False    108781
True       2429
Name: Proxy-Server, dtype: int64


##### Tests

In [10]:
df_relevante_iids.info

<bound method DataFrame.info of             iid
0     157982720
1     100425731
2     107233288
3     137986060
4     107233293
...         ...
1757  158031844
1758  100319212
1759  101253103
1760  100466672
1761  100687856

[1762 rows x 1 columns]>